## Overview
This notebook is used to generate various test vectors to support FALCON Digital Signal Processing unit tests. Vectors include: Finite Impulse Response (FIR) filter coefficients, raw complex input data, and raw complex output data.

## Constants
This notebook supports configurable behavior through environment variables, which in turn modify the notebook-level constants specified here.

In [ ]:
from fractions import Fraction
import logging
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import os
from scipy import signal
import struct

In [ ]:
FREQ_SHIFT_STR = os.environ.get('FREQ_SHIFT', '0')
INPUT_SAMPLE_RATE_STR = os.environ.get('INPUT_SAMPLE_RATE', '1000000')
OUTPUT_SAMPLE_RATE_STR = os.environ.get('OUTPUT_SAMPLE_RATE', '650000')
NUM_OUTPUT_SAMPLES_STR = os.environ.get('NUM_OUTPUT_SAMPLES', '1000000')
VERBOSE_LOGGING_STR = os.environ.get('VERBOSE', '0')
OUTPUT_FILE_BASE_NAME = os.environ.get('OUT_FILE_NAME', 'polyphase_test_vector')
GENERATE_RAMP_FILE_STR = os.environ.get('RAMP_OUTPUT', '0')

# convert from the string environment variables to other, more convenient, representations
FREQ_SHIFT = np.array([FREQ_SHIFT_STR], dtype=np.float64)
INPUT_SAMPLE_RATE = np.array([INPUT_SAMPLE_RATE_STR], dtype=np.float64)
OUTPUT_SAMPLE_RATE = np.array([OUTPUT_SAMPLE_RATE_STR], dtype=np.float64)
NUM_OUTPUT_SAMPLES = int(NUM_OUTPUT_SAMPLES_STR)
GENERATE_RAMP_FILE = int(GENERATE_RAMP_FILE_STR)

INPUT_VEC_FILE_NAME = OUTPUT_FILE_BASE_NAME + "_x.bin"
OUTPUT_VEC_FILE_NAME = OUTPUT_FILE_BASE_NAME + "_y.bin"
FILTER_COEFFS_FILE_NAME = OUTPUT_FILE_BASE_NAME + ".filter_coeffs.txt"
    
VERBOSE_LOGGING = True
if VERBOSE_LOGGING_STR is not "1":
    VERBOSE_LOGGING = False

## Logging
Use the Python *logging* module for basic logging. Configure the logging level based on the provided command-line arguments.

In [ ]:
logging_level = logging.INFO
if VERBOSE_LOGGING:
    logging_level = logging.DEBUG
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging_level)

In [ ]:
logging.info("####################")
logging.info("# FREQ_SHIFT              = %f" % (FREQ_SHIFT))
logging.info("# INPUT_SAMPLE_RATE       = %f" % (INPUT_SAMPLE_RATE))
logging.info("# OUTPUT_SAMPLE_RATE      = %f" % (OUTPUT_SAMPLE_RATE))
logging.info("# NUM_OUTPUT_SAMPLES      = %u" % (NUM_OUTPUT_SAMPLES))
logging.info("# RAMP_OUTPUT             = %u" % (GENERATE_RAMP_FILE))
logging.info("# INPUT_FILE_NAME         = %s" % (INPUT_VEC_FILE_NAME))
logging.info("# OUTPUT_FILE_NAME        = %s" % (OUTPUT_VEC_FILE_NAME))
logging.info("# FILTER_COEFFS_FILE_NAME = %s" % (FILTER_COEFFS_FILE_NAME))
logging.info("####################")

## Design a Low Pass Filter
The objective is to use the *upfirdn* function so we need to convert the decimal resampling ratio into a fraction. From there, we can design a Finite Impulse Response (FIR) filter that meets our needs.

See the following excerpt from **Boaz Porat, A Course in Digital Signal Processing**:
> "The low-pass filter performs both interpolation of the expanded signal
    and antialiasing. If the sample rate is to be increased, then p > q. THe
    low pass filter should then have a cutoff frequency of pi/p. If the
    sampling rate is to be decreased, then p < q. The low-pass filter should
    then have a cutoff frequency pi/q. IN this case the filter will
    eliminate a part of the signals frequency contents of the original
    bandwidth if its original bandwidth is higher than pip/q. Thus, the
    sampling rate conversion filter should always have a cutoff frequency
    of pi/max{p,q}."

In [ ]:
logging.info("Input Sample Rate: %d Hz" % (INPUT_SAMPLE_RATE))
logging.info("Output Sample Rate: %d Hz" % (OUTPUT_SAMPLE_RATE))

resampling_ratio = OUTPUT_SAMPLE_RATE / INPUT_SAMPLE_RATE
logging.info("Resampling Ratio %f" % (resampling_ratio))
ratio = Fraction("%.12f" % (resampling_ratio)).limit_denominator()
p = ratio.numerator
q = ratio.denominator
pqmax = max(p, q)
logging.info("Resampling Ratio: %d/%d, pqmax=%d" % (p, q, pqmax))

if int(p) != 1 or q != 1:
    
    # cutoff frequency of the lowpass filter at the high (upsampled) rate
    cutoff_freq = 1 / 2 / pqmax
    filter_order = 2 * 10 * pqmax + 1
    filter_delay = int((filter_order - 1) / p / 2)
        
    logging.info("Resampling required; computed cutoff frequency: %.2f" % (cutoff_freq))
    
else:
    
    # cutoff frequency of the lowpass filter at the high (upsampled) rate
    cutoff_freq = 0.25
    filter_order = 2 * 10 * pqmax + 1
    filter_delay = int((filter_order - 1) / p / 2)
    
    logging.info("No resampling required; computed cutoff frequency: %.2f" % (cutoff_freq))

logging.info("Cutoff Freq: %f, Filter Order: %u Filter Delay: %u" % (cutoff_freq, filter_order, filter_delay))

filter_coeffs = float(p) * signal.firls(filter_order, [0, 2.0 * cutoff_freq, 2.0 * cutoff_freq, 1.0], [1.0, 1.0, 0.0, 0.0])
filter_coeffs = filter_coeffs * signal.kaiser(filter_order, beta=5)
    
filter_coeffs_file = open(FILTER_COEFFS_FILE_NAME, 'w')
for coeff in filter_coeffs:
    filter_coeffs_file.write("%0.12f %0.12f\n" % (coeff.real, coeff.imag))
    
filter_coeffs_file.close()

logging.info("Write filter coefficients to %s" % (FILTER_COEFFS_FILE_NAME))

# visualize the filter response
w, h = signal.freqz(filter_coeffs)
fig = plt.figure()
plt.title('Digital filter frequency response')
ax1 = fig.add_subplot(111)

plt.plot(w, 20 * np.log10(abs(h)), 'b')
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [rad/sample]')

ax2 = ax1.twinx()
angles = np.unwrap(np.angle(h))
plt.plot(w, angles, 'g')
plt.ylabel('Angle (radians)', color='g')
plt.grid()
plt.axis('tight')
plt.show()

## Create a Complex Sinusoid

In [ ]:
## for fractional upsampling put the test tone closer to DC so that there
#  is more definition to the graphed original signal
us_test_tone_freq = (INPUT_SAMPLE_RATE * 0.3) / 2

# for fractional downsampling put the test tone at the outer edge of the
#  expected passband in order to verify that the filter is working
#  correctly and does not attenuate the input signal too badly
ds_test_tone_freq = (OUTPUT_SAMPLE_RATE * 0.4) / 2

# pick the minimum value
if us_test_tone_freq < ds_test_tone_freq:
    test_tone_freq = float(us_test_tone_freq)
    logging.info("Selecting 'us_test_tone_freq'=%.2f Hz" % (test_tone_freq))
else:
    test_tone_freq = float(ds_test_tone_freq)
    logging.info("Selecting 'ds_test_tone_freq'=%.2f Hz" % (test_tone_freq))
    
# adjust if we're not actually resampling
if p == 1 and q == 1:
    test_tone_freq = float(INPUT_SAMPLE_RATE * 0.1)
    logging.info("Detected no rsampling case; adjusting test tone frequency")
    
logging.info("Test tone freq=%u Hz" % (test_tone_freq))

SIGNAL_SCALE = 1024
NUM_INPUT_SAMPLES = int(NUM_OUTPUT_SAMPLES * q / p)
t = np.arange(0, NUM_INPUT_SAMPLES, dtype=np.float64)
print("number of mega bytes for 't' array: " + str(t.nbytes / 1024.0 / 1024.0))

if abs(FREQ_SHIFT) > 1:
    
    logging.info("Frequency shift required; computing 't' array")
    
    # compute an index, in floating point, which corresponds to a
    #  complete rotation
    rollover_idx_float = INPUT_SAMPLE_RATE / abs(FREQ_SHIFT)
    logging.info("Calculated float rollover index %.16f" % (rollover_idx_float))

    while rollover_idx_float < 1e5:
        rollover_idx_float = rollover_idx_float * 10.0

    logging.info("Adjusted float rollover index to %.16f" % (rollover_idx_float))
    
    rollover_idx = np.round(rollover_idx_float)
    logging.info("Computed max integer rollover index %u" % (rollover_idx))

    for ii in range(NUM_INPUT_SAMPLES):
        t[ii] = ii % rollover_idx

else:
    logging.info("No frequency shift required...")

# ceate the input test tone and convert to a complex array
x = SIGNAL_SCALE * np.exp(1j * 2 * np.pi * t * float(test_tone_freq/INPUT_SAMPLE_RATE))
x = x.astype(np.complex64)
logging.info("Input vector created")

# add a small amount of noise
#noise_power = 0.001 * INPUT_SAMPLE_RATE / 2
#x += np.random.normal(scale=np.sqrt(noise_power), size=t.shape)

if p == 1 and q == 1:
    
    logging.info("No resampling required; applying frequency shift...")
    
    # if not resampling, then just frequency shift
    angular_freq = 2.0 * np.pi * (FREQ_SHIFT/INPUT_SAMPLE_RATE)
    logging.info("Computed angular freq:%.16f" % (angular_freq))
    
    fs_vec = np.zeros(len(x), dtype=np.complex64)
    fs_vec = np.cos(angular_freq * t) + 1j * np.sin(angular_freq * t)
    y = x * fs_vec
        
    logging.info("Frequency shifted output from %d Hz to %d Hz" % (test_tone_freq, test_tone_freq + FREQ_SHIFT))
    
else:
    
    # resample using the computed values
    y = signal.upfirdn(filter_coeffs, x, p, q)

    logging.info("Generated %u resampled samples from %u input samples" % (len(y), len(x)))
    
logging.info("Input vector length=%u, Output vector length=%u" % (len(x), len(y)))

### Save the Input and Output Vectors

In [ ]:
if GENERATE_RAMP_FILE:
    logging.info("Generating RAMP file output")
else:
    logging.info("Saving Original and Resampled vectors")

# save the input "x" vector file
input_vec_file = open(INPUT_VEC_FILE_NAME, 'wb')

num_samples_processed = 0
for val in x:
    
    if GENERATE_RAMP_FILE:
        real_val = num_samples_processed % 128
        imag_val = -(num_samples_processed % 128)
    else:
        real_val = int(val.real)
        imag_val = int(val.imag)
    
    # pack as little endian format using shorts (int16_t)
    input_vec_file.write(struct.pack('<2h', real_val, imag_val))
    
    num_samples_processed += 1
    
input_vec_file.close()

# save the output "y" vector file
output_vec_file = open(OUTPUT_VEC_FILE_NAME, 'wb')

num_samples_processed = 0
for val in y:
    
    if GENERATE_RAMP_FILE:
        real_val = num_samples_processed % 128
        imag_val = -(num_samples_processed % 128)
    else:
        real_val = int(val.real)
        imag_val = int(val.imag)
    
    # pack as little endian format using shorts (int16_t)
    output_vec_file.write(struct.pack('<2h', real_val, imag_val))
    
    num_samples_processed += 1
    
output_vec_file.close()

### Time Series Plot
Plot the input and resampled time series data to visually verify that it has been resampled as expected.

In [ ]:
NUM_INPUT_SAMPLES_TO_PLOT = 512
if NUM_INPUT_SAMPLES_TO_PLOT > len(x):
    NUM_INPUT_SAMPLES_TO_PLOT = len(x)

NUM_OUTPUT_SAMPLES_TO_PLOT = int(NUM_INPUT_SAMPLES_TO_PLOT * p / q)

logging.info("Plotting %u input and %u output samples" % (NUM_INPUT_SAMPLES_TO_PLOT, NUM_OUTPUT_SAMPLES_TO_PLOT))

resampled_t = np.arange(0, len(y)) * q / p - filter_delay

plt.figure(figsize=(20, 16))
plt.subplot(2,1,1)
plt.plot(t[:NUM_INPUT_SAMPLES_TO_PLOT], x[:NUM_INPUT_SAMPLES_TO_PLOT].real, '-bo')
plt.plot(resampled_t[:NUM_OUTPUT_SAMPLES_TO_PLOT], y[:NUM_OUTPUT_SAMPLES_TO_PLOT].real, '-xr')
plt.xlabel('t')
plt.ylabel('Re x(t)')
plt.title(r'Real part of  $x(t)=e^{j 2 \pi f t}$');
plt.grid(True)

plt.subplot(2,1,2)
plt.plot(t[:NUM_INPUT_SAMPLES_TO_PLOT], x[:NUM_INPUT_SAMPLES_TO_PLOT].imag, '-bo')
plt.plot(resampled_t[:NUM_OUTPUT_SAMPLES_TO_PLOT], y[:NUM_OUTPUT_SAMPLES_TO_PLOT].imag, '-xr')
plt.xlabel('t')
plt.ylabel('Im x(t)')
plt.title(r'Imaginary part of  $x(t)=e^{j 2 \pi f t}$');
plt.grid(True)

plt.tight_layout()
plt.show()

### Spectrogram Plot

In [ ]:
FFT_SIZE = 1024
plt.figure(figsize=(16, 10))

plt.subplot(4,1,1)
Pxx, freqs, bins, im = plt.specgram(x[:FFT_SIZE*32], NFFT=FFT_SIZE, Fs=INPUT_SAMPLE_RATE)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

plt.subplot(4,1,2)
f = np.abs(np.fft.fftshift(np.fft.fft(x[:FFT_SIZE*32], FFT_SIZE)))
f = 10 * np.log10(f)
plt.plot(f)
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD (dB)')
plt.grid(True)

plt.subplot(4,1,3)
Pxx, freqs, bins, im = plt.specgram(y[filter_delay:FFT_SIZE*32], NFFT=FFT_SIZE, Fs=OUTPUT_SAMPLE_RATE)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

plt.subplot(4,1,4)
f = np.abs(np.fft.fftshift(np.fft.fft(y[filter_delay:FFT_SIZE*32], FFT_SIZE)))
f = 10 * np.log10(f)
plt.plot(f[1:])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD dB')
plt.grid(True)

plt.tight_layout()
plt.show()